In [1]:
import pandas as pd

# Load the Parquet file into a DataFrame
df = pd.read_parquet('data/preprocessed_data.parquet')

# df.to_pickle('data/preprocessed_data.pkl')
# Display the first few rows of the DataFrame
# print(df.head(3))


In [2]:
df.shape

(831, 2)

In [6]:
df.tail()

,Context,Response
826,when im not cutting im drinking when i try hea...,[it sounds like you are feeling helpless and o...
827,when im sad and alone i want to cut myself,[try doing something physical if you havent ex...
828,whenever i leave my girlfriend i get panic att...,[im sorry to hear that you are currently exper...
829,whenever i sing along with a song that i like ...,[well have you told him similar descriptions o...
830,why am i attracted to older men,[what a wonderful questiongood for you on clea...


In [7]:
df.head()

,Context,Response
0,a few years ago i was making love to my wife w...,[first step always is to do a medical rule out...
1,a lot of times i avoid situations where i am t...,[hello and thank you for your question first i...
2,a year ago the love of my life left me and nev...,[who takes care of your son is a significant p...
3,about 3 years ago or so i was skinny but i was...,[hey i am so impressed with your efforts to as...
4,about 5 months ago my ex left without fully ex...,[since you realize your sense of trust was bro...


In [2]:
df.to_json(orient='records', lines=True)

'{"Context":"a few years ago i was making love to my wife when for no known reason i lost my erection \\r\\n\\r\\nnow im in my early 30s and my problem has become more and more frequent  this is causing major problems for my ego and its diminishing my self esteem this has resulted in ongoing depression and tearing apart my marriage \\r\\n\\r\\ni am devastated and cannot find a cause for these issues i am very attracted to my wife and want to express it in the bedroom like i used to \\r\\n\\r\\nwhat could be causing this and what can i do about it","Response":["first step always is to do a medical rule out so that youre sure the problem is psychological and emotion based not a medical condition which requires care and attentionif you are medically clear in the reasons for losing your erection then reflect on what may be creating a loss in confidence in either who you are and what youre doing with your life or whether your wife has these sort of problems within herselfoften a problem tra

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [19]:
record = df.iloc[50].to_dict()
record

{'Context': 'how do i make myself happy without the people who made me happy now that theyre gone i feel sad its been two months now but i seem to be unable to stay okay and independent',
 'Response': array(['hello so someone made you happy but is no longer around i am sure that feels devastating as a counselor i usually have to ponder this kind of thought because it lends itself to suggesting that happiness comes from the external rather than the internal side of a persons life happiness is formed within ourselves all the external trappings of happiness are usually quite fleeting and temporary even relationships fo not last forever if we get to a place where we are reliant and dependent on those around us to create happiness in our lives then we lose the opportunities for us to do it ourselves we are entirely capable of finding ways to be happy sometimes this entails working out issues that are holding us back from reaching a place of peace or resolution to a specific problemwhat i wo

In [20]:
# Check the data type of the 'Context' field
print(df['Context'].dtype)

object


In [4]:
# Filter rows where 'Context' and 'Response' are non-empty
non_empty_rows = df[df['Context'].str.strip().astype(bool) & df['Response'].map(lambda x: len(x) > 0)]

# Display the filtered DataFrame
non_empty_rows.head()
print(f"Total non-empty rows: {len(non_empty_rows)}")

Total non-empty rows: 995


In [16]:
import numpy as np

# Flatten the Response column if it contains lists
df['Response_flat'] = df['Response'].apply(lambda x: ', '.join(x) if isinstance(x, (list, np.ndarray)) else x)

# Check for duplicates in the flattened Response column
duplicates = df[df.duplicated(subset='Response_flat', keep=False)]

# Display duplicate rows
print(duplicates.head())
print(f"Total duplicate rows: {len(duplicates)}")

                                               Context  \
48   how do i ever trust another woman i have found...   
49   how do i ever trust another woman i have found...   
72   i am currently suffering from erectile dysfunc...   
73   i am currently suffering from erectile dysfunc...   
121  i feel really uncomfortable when i have people...   

                                              Response  \
48   [im sorry to hear about being taken advantage ...   
49   [im sorry to hear about being taken advantage ...   
72   [hi first and foremost i want to acknowledge y...   
73   [hi first and foremost i want to acknowledge y...   
121  [you can learn lots of skillstechniques to fee...   

                                         Response_flat  
48   im sorry to hear about being taken advantage o...  
49   im sorry to hear about being taken advantage o...  
72   hi first and foremost i want to acknowledge yo...  
73   hi first and foremost i want to acknowledge yo...  
121  you can lear

In [21]:
# Convert Context to string if it's not already
df['Context'] = df['Context'].astype(str)

# Check for duplicates in the Context column
context_duplicates = df[df.duplicated(subset='Context', keep=False)]

# Display duplicate rows
print(context_duplicates.head())
print(f"Total duplicate rows: {len(context_duplicates)}")

Empty DataFrame
Columns: [Context, Response, Response_flat, Context_flat]
Index: []
Total duplicate rows: 0


In [23]:
# Preprocess the 'Context' column
df['Context_clean'] = df['Context'].str.lower().str.strip()
df['Context_clean'] = df['Context_clean'].replace(r'\s+', ' ', regex=True)  # Remove multiple spaces

# Check for duplicates in the cleaned 'Context' column
context_duplicates_cleaned = df[df.duplicated(subset='Context_clean', keep=False)]
context_duplicates_cleaned.head()
print(f"Total duplicate rows (cleaned): {len(context_duplicates_cleaned)}")


Total duplicate rows (cleaned): 328


In [13]:
import uuid

def add_ids_to_original_data(original_df):
    # Initialize a new list to store results with IDs
    data_with_ids = []

    # Iterate through the original DataFrame and assign unique IDs
    for index, row in original_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Generate a unique ID for this record
        unique_id = uuid.uuid4().hex[:10]  # Generate a shorter unique ID
        
        # Add ID to the record
        data_with_ids.append({
            'ID': unique_id,
            'Context': context,
            'Response': response
        })

    # Convert to DataFrame
    df_with_ids = pd.DataFrame(data_with_ids)

    return df_with_ids

# Apply the ID assignment function
original_df_with_ids = add_ids_to_original_data(df)

# Example output inspection
original_df_with_ids.head()


,ID,Context,Response
0,30916995b7,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,8cd206d284,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,9a7102ef25,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,ec62b091bc,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,84926b95d3,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Start Here

In [4]:
df1 = df.copy()

In [6]:
# Drop duplicate questions while keeping the first occurrence of each
df_no_duplicates = df.drop_duplicates(subset=['Context', 'Response'])

# Group by 'context' and aggregate 'response' into lists
df_grouped = df_no_duplicates.groupby('Context')['Response'].agg(list).reset_index()

# Separate contexts with multiple and single responses
df_multiple_answers = df_grouped[df_grouped['Response'].map(len) > 1]
df_single_answers = df_grouped[df_grouped['Response'].map(len) == 1]

# Flatten the single responses lists
df_single_answers['Response'] = df_single_answers['Response'].str[0]

C:\Users\USER\AppData\Local\Temp\ipykernel_20516\1460053985.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_answers['Response'] = df_single_answers['Response'].str[0]


In [7]:
df_no_duplicates.shape

(2752, 2)

In [8]:
df_grouped.shape

(995, 2)

In [9]:
df_multiple_answers.shape

(557, 2)

In [11]:
df_single_answers.shape

(438, 2)

In [12]:
# Combine the DataFrames if needed
df_processed = pd.concat([df_multiple_answers, df_single_answers], ignore_index=True)

In [13]:
df_processed.shape

(995, 2)

In [16]:
df_processed.nunique()

TypeError: unhashable type: 'list'

In [12]:
df2 = df1.copy()

In [13]:
# This will keep the first occurrence of each duplicate row
df_no_duplicates = df1.drop_duplicates()

In [17]:
# Group by the 'context' (question) column and aggregate the 'response' (answer) column
df_grouped = df_no_duplicates.groupby('Context')['Response'].agg(list).reset_index()

# Filter to keep only questions with multiple answers
df_multiple_answers = df_grouped[df_grouped['Response'].map(len) > 1]

In [18]:
df_single_answers = df_grouped[df_grouped['Response'].map(len) == 1]
df_single_answers['Response'] = df_single_answers['Response'].map(lambda x: x[0])

# Combine multiple and single answer dataframes
df_processed = pd.concat([df_multiple_answers, df_single_answers])

C:\Users\USER\AppData\Local\Temp\ipykernel_20516\1605488034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_answers['Response'] = df_single_answers['Response'].map(lambda x: x[0])


In [24]:
df_processed.isna().sum()

Context     0
Response    0
dtype: int64

In [19]:
df_multiple_answers.head(10)

,Context,Response
3,A few years ago I was making love to my wife w...,[First step always is to do a medical rule out...
4,A few years ago I was making love to my wife w...,[First step always is to do a medical rule out...
7,"A lot of times, I avoid situations where I am ...","[Hello, and thank you for your question. First..."
9,"A year ago, the love of my life left me and ne...","[Who takes care of your son, is a significant ..."
11,"About 3 years ago or so I was skinny, but I wa...",[Hey! I am so impressed with your efforts to ...
12,About 5 months ago my ex left without fully ex...,[since you realize your sense of trust was bro...
14,"About two and a half months ago, I met a woman...",[That sounds really heartbreaking. It is tough...
15,"About two and a half months ago, I met a woman...",[Love doesn't hurt.Your description of the rel...
16,"After 25 years, I fell in love for the first t...",[Ouch. Losing someone you love hurts so much. ...
17,"After 40 years of being straight, how could I ...",[What most people don't know is that sexual pr...


In [23]:
df_single_answers.shape

(438, 2)

In [22]:
df_multiple_answers.shape

(557, 2)

In [28]:
df_no_duplicates.shape

(2752, 2)